<a href="https://colab.research.google.com/github/maberf/colabs/blob/main/portfolioold2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

********************************************************************************

PORTFOLIO INVESTOR CODE - MAIN

********************************************************************************

In [ ]:
# Code to handle portfolio and assets in an Google Sheets file
# This is an integrated system with Google Sheets and python code working togheter
#
# GOOGLE SHEETS
# portfolio  - Google Sheets file name with all data on sheets inside
#
# Sheets:
# parameters - general parameters inputs to used in portfolio calculations
# portfolio_input - protfolio inputs values
# stockbr - BR stocks output factors from historical series in BRL
# reitbr - BR REITs (FIIs) output actors from historical series in BRL
# stockus - US stocks factors output from historical series in USD
# etfus - US ETFs factors output from historical series in USD
# reitus - US REITs factors output from historical series in USD
# portfolio - portfolio assets key numbers output in BRL (US assets are converted to BRL)
# stockbr_info - stockbr assets fundamentalist data output
# reitbr_info - reitbr assets fundamentalists data output
# stockus_info - stock_us assets fundamentalists data output
# quotes - sheet to consolidate quotation from Google Finance and the values from sheets stockbr, stockus, etfus and reitus
#
# QUOTES SHEET CONSOLIDATION
# In sheet "quotes" you can create a cell with tickers quotations from Google Finance and consolidade assets factors and assets infos
# The user can calulate multiples such as asset discounted cash flow (DCF) to estimate estimate real value
# in quotes you can combine inputs and outputs from ohter sheets readings also
# "quotes" sheet can be customized according user desire and multiples
#
# Data Sources:
# Google Finance - To real time quotations in quotes sheet
# Yahoo Finance - To assets series, BR assets needs '.SA" added to the ticker. Yahoo Finance provide US assets fundamentalists data, but not in case o BR assets
# Fundamentus - Scraping informations from https://www.fundamentus.com.br to BR stocks
# Funds Explorer - Copy data from https://www.fundsexplorer.com.br/ranking and paste only values data in an excel spreadsheet
# Investing.com - IFIX history series csv file with time desired period (2y, 3y, etc.) from https://br.investing.com/indices/bm-fbovespa-real-estate-ifix-historical-data
# Funds Explorer and Ifix Historical files should in a Google Drive directory which the the path is on code, adjust the path on code if needed
#
# IMPORTANT
# Create and save .py specific package files in the correct path to the this main code
# Make activities related to data source before run the code
# IFIX series should be higher than adjusted in Period parameter, it is suggested last working day plus period plus 12 months beginnig
# Verifiy data of source files files in case of code running problems
# During the code running it is necessary to allow access to Google, necessary code access Google Drive where files are located
# Check key moments in code, use display and prints commented in code if needed
# - portfolio dataframe creation
# - parameters and assets lists generation
# - historical data series readings
# - portfolio dataframe assembling
# - maximum and target sharpe numbers portfolio adding
# - assets infos readings and writing on sheets
# You can export the sheets to excel, essentially the sheets portfolio and quotes to manage your specific assets and strategies
# Fundamentalist infos do not change in short time, so InfoReading? = "Yes" to the first running, after change to "No"

********************************************************************************
LIBRARY IMPORTS
********************************************************************************

In [ ]:
# standard packages imports
import sys
import os

from google.colab import auth
from google.auth import default
import gspread
from google.colab import drive

import pandas as pd
import numpy as np
from scipy.optimize import minimize
import yfinance as yf
import datetime as dt


In [ ]:
# drive mount and path to modules
drive.mount('/content/drive', force_remount=True)
sys.path.append('/content/drive/My Drive/ColabNotemodules')
# print('Current sys.path:', sys.path)
# To check the modules are being reacheble by this code (linux command)
!ls /content/drive/My\ Drive/ColabNotemodules/invest

In [ ]:
# specific (modules) packages import functions
# BR and US data series reading and US assets info reading
from invest.yahoofinance import yfSeries, yfStockUsData, yfSanitizeInfos
# BR reits (FIIs) series and info readings
from invest.reitbr import ifix, reitbrData
# converts USD columns to BRL
from invest.usdbrl import usdToBrl
# calculate assets factors - min, max return, risk and beta
from invest.assetsfactors import assetsFactors
# calculate portfolio performance
from invest.portfolioperformance import portfolioPerformance, maxSharpeWeights, targetSharpeWeights
# scraper fundamentus to BR stocks infos
from invest.fundamentus import scraperFundamentus

********************************************************************************
GOOGLE SHEETS INPUTS READINGS
********************************************************************************

In [ ]:
# Google Sheets conection
auth.authenticate_user()
creds, _ = default()
gc = gspread.authorize(creds)
sh = gc.open('portfolio')

In [ ]:
# generic reading function
def read_sheet(tab):
    ws = sh.worksheet(tab)
    df = pd.DataFrame(ws.get_all_records())
    return df

In [ ]:
# generic parameters reading
parameters = read_sheet('parameters')

# extracting float values
risk_free = float(parameters.loc[parameters['Parameter'] == 'RiskFree', 'Value'].values[0]) / 100
target_sharpe = float(parameters.loc[parameters['Parameter'] == 'TargetSharpe', 'Value'].values[0]) / 100
adjusting_factor = float(parameters.loc[parameters['Parameter'] == 'AdjustingFactor', 'Value'].values[0]) / 100000

# extracting string
period = str(parameters.loc[parameters['Parameter'] == 'Period', 'Value'].values[0])
info_reading = str(parameters.loc[parameters['Parameter'] == 'InfoReading?', 'Value'].values[0])

print(f'risk_free =', risk_free,'  period=' ,period, '  target_sharpe =', target_sharpe,
      '  adjusting_factor = ', adjusting_factor, '  info_reading =', info_reading)

In [ ]:
# portfolio input reading
portfolio = read_sheet('portfolio_input')

# adjustig dataframe columns data
portfolio = portfolio[portfolio['Ticker'] != '']
portfolio['RetE%'] = portfolio['RetE%'].astype(float) / 10
portfolio['W'] = portfolio['W'].astype(float) / 1000
portfolio['Limit'] = portfolio['Limit'].astype(float)

display(portfolio)

# if currency = 'BRL', add '.SA' to ticker name
# portlist_df is a temporary dataframe only to generate adequate list
portlist_df = portfolio[['Curr','Ticker']].copy()
portlist_df['Ticker'] = portlist_df.apply(
    lambda row: f"{row['Ticker']}.SA" if row['Curr'] == 'BRL' else row['Ticker'],
    axis=1
)
# display(portlist_df)

#
port_ticker = portlist_df['Ticker'].dropna().tolist()
port_ticker = [t for t in port_ticker if str(t).strip() != ""]

# '.SA' adding and replace IBOV to '^BVSP^ and USBBRL to USDBRL=X
# port_ticker = [nome + '.SA' for nome in port_ticker]
port_ticker = ['^BVSP' if nome == 'IBOV.SA' else nome for nome in port_ticker]
port_ticker = ['USDBRL=X' if nome == 'USDBRL.SA' else nome for nome in port_ticker]
print(f'port_ticker =', port_ticker)

# expected return list
port_expret = portfolio['RetE%'].tolist()
print(f'port_expret =', port_expret)

# weights list
port_weight = portfolio['W'].tolist()
print(f'port_weight =', port_weight)

# limits list
port_limit = portfolio['Limit'].tolist()
# limits adjusted by factor
port_limits = [round(x * adjusting_factor, 3) for x in port_limit]
print(f'port_limit =', port_limit)
print(f'port_limits(adjusted) =', port_limits)


In [ ]:
# tickers reading
ticker_df = read_sheet("tickers_input")

# remove linhas completamente vazias
ticker_df = ticker_df.dropna(how="all")

def get_tickers(col):
    return ticker_df[col].dropna().tolist()

# lists creation
stockbr_ticker = get_tickers('StockBr')
reitbr_ticker  = get_tickers('ReitBr')
stockus_ticker = get_tickers('StockUs')
etfus_ticker   = get_tickers('EtfUs')
reitus_ticker  = get_tickers('ReitUs')

# clean lists empty spaces
stockbr_ticker = [item for item in stockbr_ticker if item is not None and item != '']
reitbr_ticker = [item for item in reitbr_ticker if item is not None and item != '']
stockus_ticker = [item for item in stockus_ticker if item is not None and item != '']
etfus_ticker   = [item for item in etfus_ticker if item is not None and item != '']
reitus_ticker  = [item for item in reitus_ticker if item is not None and item != '']

# conformation lists to historical Yahoo Finance series download preparation
# '.SA' adding and replace IBOV to '^BVSP^ to StockBr list
stockbr_ticker = [nome + '.SA' for nome in stockbr_ticker]
stockbr_ticker = ['^BVSP' if nome == 'IBOV.SA' else nome for nome in stockbr_ticker]
# '.SA' adding and exclude IFIX from ReitBr list
reitbr_ticker = [nome + '.SA' for nome in reitbr_ticker]
# reitbr_ticker = [nome for nome in reitbr_ticker if nome != 'IFIX.SA']
# replace SP500 by ^GSPC and USDBRL by USDBRL=X on lists StockUS and EtfUS
stockus_ticker = ['^GSPC' if nome == 'SP500' else nome for nome in stockus_ticker]
stockus_ticker = ['USDBRL=X' if nome == 'USDBRL' else nome for nome in stockus_ticker]
etfus_ticker = ['^GSPC' if nome == 'SP500' else nome for nome in etfus_ticker]
etfus_ticker = ['USDBRL=X' if nome == 'USDBRL' else nome for nome in etfus_ticker]
# replace USDBRL by USDBRL=X on list ReistUS
reitus_ticker = ['USDBRL=X' if nome == 'USDBRL' else nome for nome in reitus_ticker]

# print checking
print(f'stockbr_ticker =', stockbr_ticker)
print(f'reitbr_ticker =', reitbr_ticker)
print(f'stockus_ticker =', stockus_ticker)
print(f'etfus_ticker =', etfus_ticker)
print(f'reitus_ticker =', reitus_ticker)

********************************************************************************
DATA SERIES READINGS
********************************************************************************

In [ ]:
# READING FROM DATA SERIES SOURCES

# series reading from Yahoo Finance
stockbr_series = yfSeries(stockbr_ticker, period=period, pricetype='Close')
reitbr_series = yfSeries(reitbr_ticker, period=period, pricetype='Close')
stockus_series = yfSeries(stockus_ticker, period=period, pricetype='Close')
etfus_series = yfSeries(etfus_ticker, period=period, pricetype='Close')
reitus_series = yfSeries(reitus_ticker, period=period, pricetype='Close')
port_series = yfSeries(port_ticker, period=period, pricetype='Close')
# display(stockbr_series)

In [ ]:
# read ifix historical series and upload in a dataframe
# use from invest.reitbr import ifix
ifix_series = ifix ('/content/drive/MyDrive/Financas/history.csv')
# display(ifix_series)
# replace reitbr_series dataframe by ifix_series values by index key (date)
reitbr_series.update(ifix_series)
# display(reitbr_series)

In [ ]:
# timeline harmonization, same period of time to all series
common_idx = (
    stockbr_series.index
    .intersection(reitbr_series.index)
    .intersection(stockus_series.index)
    .intersection(etfus_series.index)
    .intersection(reitus_series.index)
    .intersection(port_series.index)
)

# Reindex and filling (without warnings)
stockbr_series = stockbr_series.reindex(common_idx).ffill()
reitbr_series = reitbr_series.reindex(common_idx).ffill()
stockus_series = stockus_series.reindex(common_idx).ffill()
etfus_series = etfus_series.reindex(common_idx).ffill()
reitus_series = reitus_series.reindex(common_idx).ffill()
port_series = port_series.reindex(common_idx).ffill()
# display(stockbr_series)

In [ ]:
# converts portfolio series USD columns to BRL
# use from invest.usdbrl import usdToBrl
port_series = usdToBrl(port_series, stockus_series)
port_series = usdToBrl(port_series, etfus_series)
port_series = usdToBrl(port_series, reitus_series)
# display(port_series)

In [ ]:
# calculate asstes factors
# use from invest.assetsfactors import assetsFactors
stockbr_factors = assetsFactors(stockbr_series)
reitbr_factors = assetsFactors(reitbr_series)
stockus_factors = assetsFactors(stockus_series)
etfus_factors = assetsFactors(etfus_series)
reitus_factors = assetsFactors(reitus_series)
port_factors = assetsFactors(port_series)
# display(port_factors)
# display(portfolio)

********************************************************************************
PORTFOLIO DATAFRAME ASSEMBLING
********************************************************************************

In [ ]:
# portfolio dataframe assembling
portfolio = pd.merge(portfolio, port_factors, on='Ticker', how='inner')
display(portfolio)

In [ ]:
# PORTFOLIO PERFORMANCE
# uses from invest.portfolioperformance import portfolioPerformance, maxSharpeWeights, targetSharpeWeights

In [ ]:
riskfree = risk_free / 100 # Risk Free is indicated in percentage

In [ ]:
# daily variation
port_variation = port_series.pct_change()
# portfolio covariance calculation
port_covariance = port_variation.cov()*252
# convert covariance dataframe in numpy matrix
cov = port_covariance.values

In [ ]:
# weights array
weights = [values / 100 for values in port_weight]

In [ ]:
# extracting historical real returns
mu = portfolio['RetH%'].values / 100

In [ ]:
# porfolio Historical Return, Risk and Sharpe calculations
portfoliorettotal, portfoliorisktotal, portfoliosharpe = portfolioPerformance(weights, mu, cov, riskfree)
# print (portfoliorettotal, portfoliorisktotal, portfoliosharpe)

In [ ]:
# portfolio adding performance columns in first register(in the same line of IBOV value index)
# other register lines being filled with zero and adequate rounds to values
# Total Historical Return adding
portfolio['RetHT%'] = [portfoliorettotal * 100] + [0] * (len(portfolio) - 1)
portfolio['RetHT%'] = portfolio['RetHT%'].round(1)
# Total Historical Risk adding
portfolio['RiskT%'] = [portfoliorisktotal * 100] + [0] * (len(portfolio) - 1)
portfolio['RiskT%'] = portfolio['RiskT%'].round(1)
# Total Beta adding
portfolio['BetaT'] = [(portfolio['W'] / 100 * portfolio['Beta']).sum()] + [0] * (len(portfolio) - 1)
portfolio['BetaT'] = portfolio['BetaT'].round(3)
# Total Historical Sharpe adding
portfolio['SharpeH%'] = [portfoliosharpe] + [0] * (len(portfolio) - 1)
portfolio['SharpeH%'] = portfolio['SharpeH%'].round(3)
# display (portfolio)

In [ ]:
# drop Min and Max
portfolio = portfolio.drop(columns=['Min', 'Max'])
# displace Expected Return to rigth, just to organization
cols = list(portfolio.columns)
i = cols.index('RetE%')
n = 4
new_order = cols[:i] + cols[i+1 : i+1+n] + [cols[i]] + cols[i+1+n:]
portfolio = portfolio[new_order]
# display (portfolio)

In [ ]:
# extracting expected returns
mu = portfolio['RetE%'].values / 100

In [ ]:
# porfolio Expected Return, Expected Risk and Expected Sharpe calculations
portfolioretexptotal, portfolioriskexptotal, portfoliosharpeexp = portfolioPerformance(weights, mu, cov, riskfree)
# print (portfolioretexptotal, portfolioriskexptotal, portfoliosharpeexp)

In [ ]:
# portfolio adding columns with Total Return, Total Risk and Sharpe values in first register(in the same line of IBOV value index)
# other register lines being filled with zero and adequate rounds to values
portfolio['RetET%'] = [portfolioretexptotal * 100] + [0] * (len(portfolio) - 1)
portfolio['RetET%'] = portfolio['RetET%'].round(1)
# Estimated Total Risk and Historic
# portfolio['RiskET%'] = [portfolioriskexptotal * 100] + [0] * (len(portfolio) - 1)
# portfolio['RiskET%'] = portfolio['RiskET%'].round(0)
portfolio['SharpeET%'] = [portfoliosharpeexp] + [0] * (len(portfolio) - 1)
portfolio['SharpeET%'] = portfolio['SharpeET%'].round(3)
# display (portfolio)

***************************************************************************
MAXIMUM AND TARGET SHARPES
***************************************************************************

In [ ]:
# MAXIMUM SHARPE calculation
res_max, w_max = maxSharpeWeights(mu, cov, riskfree)
print(res_max)
# print("Max Sharpe:", portfolioPerformance(w_max, mu, cov, riskfree)[2], "weights:", w_max)
# print("Ret Max Sharpe:", w_max)

In [ ]:
#  add Maximum Sharpe ticker weights to portfolio dataframe
portfolio['ShMaxE-W'] = w_max.round(3) * 100
# display(portfolio)

In [ ]:
# add Maximum Sharpe value to portfolio dataframe in the first line
portfolio.at[0, 'ShMaxE-W'] = round(portfolioPerformance(w_max, mu, cov, riskfree)[2], 3)
# display(portfolio)

In [ ]:
# add Sharpe Maximum Return column and the value in the first line
portfolio['ShMaxRetE%'] = [portfolioPerformance(w_max, mu, cov, riskfree)[0] *100] + [0] * (len(portfolio) - 1)
portfolio['ShMaxRetE%'] = portfolio['ShMaxRetE%'].round(1)
# display(portfolio)

In [ ]:
# TARGET SHARPE calculation as a percentage of MAXIMUM TARGET (ShMaxE-W)
target = portfolio.at[0, 'ShMaxE-W'] * target_sharpe
print("Target:", target)

# CORRECT call: pass limitsport (in fraction 0..1) as argument
res_tgt, w_tgt, info = targetSharpeWeights(mu, cov, target, riskfree, port_limits)

# Debug/output (optional)
print(res_tgt)
# print("Target result:", info)
# print("Weights:", w_tgt)
# print("Achieved Sharpe:", info.get('achieved_sharpe'))

In [ ]:
#  Add Target Sharpe ticker weights to portfolio dataframe
portfolio['ShTg-W'] = (w_tgt * 100).round(3)
# Add Target (Achieved) Sharpe value to portfolio dataframe in the first line
portfolio.at[0, 'ShTg-W'] = round(info['achieved_sharpe'], 3)
# Add Target Maximum Return column and the value in the first line
portfolio['ShTgRetE%'] = [portfolioPerformance(w_tgt, mu, cov, riskfree)[0] *100] + [0] * (len(portfolio) - 1)
portfolio['ShTgRetE%'] = portfolio['ShTgRetE%'].round(1)
display(portfolio)

***************************************************************************
ASSETS INFO READINGS
***************************************************************************

In [ ]:
# use from invest.fundamentus import scraperFundamentus
if info_reading == 'Yes':
    # FUNDAMENTUS BRAZIL STOCKS INFO READING (scraping)
    stockbr_info = scraperFundamentus(stockbr_ticker)
    # replace NaN values with None in reitbr_info before updating the sheet
    stockbr_info = stockbr_info.replace({np.nan: None})
    # display(stockbr_info)

In [ ]:
# use from invest.reitbr import reitbrData
if info_reading == 'Yes':
    # FUNDS EXPLORER REAL STATE BR INFO READING
    reitbr_info = reitbrData('/content/drive/MyDrive/Financas/fundsexplorer.xlsx')
    # list filter, exclude tickers not in real state list ()
    reitbr_ticker_adjusted = [nome.replace('.SA', '') for nome in reitbr_ticker]
    if "IFIX" in reitbr_ticker_adjusted:
        reitbr_ticker_adjusted.remove("IFIX")
    reitbr_info = reitbr_info[reitbr_info['Fundos'].isin(reitbr_ticker_adjusted)]
    reitbr_info = reitbr_info.reset_index(drop=True)
    # replace NaN values with None in reitbr_info before updating the sheet
    reitbr_info = reitbr_info.replace({np.nan: None})
    # display(reitbr_info)

In [ ]:
# use from invest.yahoofinance import yfStockUsData, yfSanitizeInfos
if info_reading == 'Yes':
    # YAHOO FINANCE STOCK US INFO READING
    stockus_info = yfStockUsData(stockus_ticker)
    # sanitize values
    stockus_info = yfSanitizeInfos(stockus_info)
    # display(stockus_info)

In [ ]:
# SPREADSHEET WRITINGS

***************************************************************************
GOOGLE SHEETS WRITINGS
***************************************************************************

In [ ]:
# Autentication in Google Docs (only once)
# auth.authenticate_user()
# creds, _ = default()
# gc = gspread.authorize(creds)

In [ ]:
# Open workbook and worksheets
wb = gc.open('portfolio')
wsport = wb.worksheet('portfolio')
wsstockbr = wb.worksheet('stockbr')
wsreitbr = wb.worksheet('reitbr')
wsstockus = wb.worksheet('stockus')
wsetfus = wb.worksheet('etfus')
wsreitus = wb.worksheet('reitus')
if info_reading == 'Yes':
    wsstockbr_info = wb.worksheet('stockbr_info')
    wsreitbr_info = wb.worksheet('reitbr_info')
    wsstockus_info = wb.worksheet('stockus_info')

In [ ]:
# Write data in the worksheets
wsport.update([portfolio.columns.values.tolist()] + portfolio.values.tolist())
wsstockbr.update([stockbr_factors.columns.values.tolist()] + stockbr_factors.values.tolist())
wsreitbr.update([reitbr_factors.columns.values.tolist()] + reitbr_factors.values.tolist())
wsstockus.update([stockus_factors.columns.values.tolist()] + stockus_factors.values.tolist())
wsetfus.update([etfus_factors.columns.values.tolist()] + etfus_factors.values.tolist())
wsreitus.update([reitus_factors.columns.values.tolist()] + reitus_factors.values.tolist())
if info_reading == 'Yes':
    wsstockbr_info.update([stockbr_info.columns.values.tolist()] + stockbr_info.values.tolist())
    wsreitbr_info.update([reitbr_info.columns.values.tolist()] + reitbr_info.values.tolist())
    wsstockus_info.update([stockus_info.columns.values.tolist()] + stockus_info.values.tolist())

In [ ]:
print("END")